In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = 'https://www.vivareal.com.br/venda/sp/campinas/apartamento_residencial/'

# Input do número total de páginas
num_paginas = int(input("Digite o número total de páginas: "))

# Lista vazia para armazenar os dados dos anúncios
data = []

# Loop pelas páginas para extrair as informações dos anúncios
for i in range(1, num_paginas + 1):
    # URL da página atual
    url_atual = url + f'?pagina={i}'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url, headers=headers)

soup = BeautifulSoup(response.text, 'html.parser')

primeiro_anuncio = soup.find('article', {'class': 'property-card__container js-property-card'})

ul_comodidades = primeiro_anuncio.find('ul', {'class': 'property-card__amenities'})

if ul_comodidades:
    comodidades = [comodidade.get('title') for comodidade in ul_comodidades.find_all('li')]
    print(comodidades)
else:
    print('Comodidades não encontradas')
    
anuncios = soup.find_all('article', {'class': 'property-card__container js-property-card'})
comodidades_unicas = set()

# Obter todas as comodidades presentes nos anúncios
for anuncio in anuncios:
    ul_comodidades = anuncio.find('ul', {'class': 'property-card__amenities'})
    if ul_comodidades:
        comodidades = [comodidade.get('title') for comodidade in ul_comodidades.find_all('li')]
        comodidades_unicas.update(comodidades)

# Criar DataFrame
df_amenities = pd.DataFrame(columns=list(comodidades_unicas))

# Preencher DataFrame com presença/ausência de cada comodidade em cada anúncio
for i, anuncio in enumerate(anuncios):
    ul_comodidades = anuncio.find('ul', {'class': 'property-card__amenities'})
    if ul_comodidades:
        comodidades = {comodidade.get('title'): True for comodidade in ul_comodidades.find_all('li')}
    else:
        comodidades = {}
    df_amenities.loc[i] = {comodidade: comodidades.get(comodidade, False) for comodidade in comodidades_unicas}
    
print(df_amenities)


Digite o número total de páginas: 10
['Piscina', 'Churrasqueira', 'Elevador', 'Aceita animais', 'Garagem', None]
    Garagem  Academia  Cozinha  Churrasqueira  Espaço gourmet  Playground  \
0      True     False    False           True           False       False   
1     False     False    False          False           False       False   
2     False      True    False           True           False       False   
3     False     False    False          False           False       False   
4     False     False    False          False           False       False   
5     False      True    False          False           False       False   
6     False     False    False          False           False       False   
7     False     False    False          False           False       False   
8     False     False    False          False           False       False   
9     False     False    False          False           False       False   
10    False      True    False          

In [3]:
url = 'https://www.vivareal.com.br/venda/sp/campinas/apartamento_residencial/'

# Input do número total de páginas
num_paginas = int(input("Digite o número total de páginas: "))

# Lista vazia para armazenar os dados dos anúncios
data = []

# Loop pelas páginas para extrair as informações dos anúncios
for i in range(1, num_paginas + 1):
    # URL da página atual
    url_atual = url + f'?pagina={i}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    response = requests.get(url_atual, headers=headers)

    soup = BeautifulSoup(response.text, 'html.parser')

    anuncios = soup.find_all('article', {'class': 'property-card__container js-property-card'})

    for anuncio in anuncios:
        # Extrair informações do anúncio
        endereco = anuncio.find('span', {'class': 'property-card__address'}).text.strip()
        area = anuncio.find('span', {'class': 'property-card__detail-area'}).text.strip()
        quartos = anuncio.find('li', {'class': 'property-card__detail-room'}).span.text.strip()
        banheiros = anuncio.find('li', {'class': 'property-card__detail-bathroom'}).span.text.strip()
        vagas = anuncio.find('li', {'class': 'property-card__detail-garage'}).span.text.strip()
        valor = anuncio.find('div', {'class': 'property-card__price'}).text.strip()

        # Extrair preço do condomínio
        condominio_tag = anuncio.find('div', {'class': 'property-card__price-details--condo'})
        if condominio_tag:
            condominio = condominio_tag.text.strip()
        else:
            condominio = '0'

        # Extrair comodidades do anúncio
        ul_comodidades = anuncio.find('ul', {'class': 'property-card__amenities'})
        if ul_comodidades:
            comodidades = [comodidade.get('title') for comodidade in ul_comodidades.find_all('li')]
        else:
            comodidades = [False] * len(df_amenities.columns) # Retorna lista com False caso não encontre comodidades

        # Salvar informações em um dicionário
        anuncio_dict = {
            'Endereço': endereco,
            'Área': area,
            'Quartos': quartos,
            'Banheiros': banheiros,
            'Vagas': vagas,
            'Valor': valor,
            'Condomínio': condominio
        }
        anuncio_dict.update({comodidade: True for comodidade in comodidades})
        data.append(anuncio_dict)

# Criar DataFrame
df = pd.DataFrame(data)

# Reordenar as colunas do DataFrame
colunas = ['Endereço', 'Área', 'Quartos', 'Banheiros', 'Vagas', 'Valor', 'Condomínio']
colunas += [coluna for coluna in df.columns if coluna not in colunas]
df = df[colunas]

# Salvar DataFrame em um arquivo Excel
df.to_excel('anuncios.xlsx', index=False)

print(df)


Digite o número total de páginas: 10
                                              Endereço Área Quartos Banheiros  \
0    Rua Jacy Teixeira Camargo, 240 - Jardim do Lag...   80       3         2   
1    Rodovia Governador Doutor Adhemar Pereira de B...  360      --        --   
2    Rua Campos do Jordão, 449 - Fundação da Casa P...   54       2         2   
3    Rua Pedro Vieira da Silva, 595 - Jardim Santa ...   77       3         2   
4       Rua João Iamarino, 3 - Taquaral, Campinas - SP   50       2         1   
..                                                 ...  ...     ...       ...   
355  Avenida Carlos de Araújo Gobbi, 220 - Vila São...   59       2         1   
356  Avenida Washington Luís, 4300 - Vila Marieta, ...   58       3         1   
357  Rua Almirante Custódio José de Melo, 351 - Jar...   60       2         1   
358  Avenida Engenheiro Antônio Francisco de Paula ...   43       2         1   
359  Rua Jacy Teixeira Camargo, 240 - Jardim do Lag...   82       3     